In [94]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from scipy.interpolate import interp1d
import pandas as pd

In [ ]:
def cte_expand(a, b, c, d, e, f, g, h, i, temp):
    power_int = np.array([a, b, c, d, e, f, g, h, i])
    T = np.array(temp)
    l = np.log10(T)
    power = np.zeros(len(T))
    for p, i in enumerate(power_int):
        power += i * 1e-6 * l**p

    print(power)

    return 10**power

In [ ]:
temps = np.linspace(4, 298, 100)
cte1 = cte_expand(-17.9081289, 67.131914, -118.809316, 109.9845997, -53.8696089, 13.30247491, -1.30843441, 0, 0, temps)
plt.plot(temps, cte1)
plt.show()

In [95]:
# Load the CSV file with proper parsing
def csv2cte(filepath):
    data = pd.read_csv(filepath, skiprows=1, delim_whitespace=True)

    # Extract relevant columns
    split_data = data.iloc[:, 0].str.split(',', expand=True)

    # Extract the relevant columns
    temperatureStr = split_data[0].values
    valueStr = split_data[2].values

    #convert to float
    temperature = temperatureStr.astype(float)
    value = valueStr.astype(float)

    return temperature, value

In [106]:
copper_cte = csv2cte('copper.csv')
silicon_cte = csv2cte('silicon.csv')

/var/folders/v8/f7ff7bzn66z3q5_fw16grwb80000gn/T/ipykernel_86717/3425989425.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(filepath, skiprows=1, delim_whitespace=True)
/var/folders/v8/f7ff7bzn66z3q5_fw16grwb80000gn/T/ipykernel_86717/3425989425.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(filepath, skiprows=1, delim_whitespace=True)


In [97]:
def lengthChangeWithTemp(material, cte_data, initL, startT, endT):
    temp = np.array(cte_data['temperature'])
    cte = np.array(cte_data['cte'])

    #interpolate the cte data
    cte_interpolated = interp1d(temp, cte, kind='linear', fill_value="extrapolate")
    temps = np.linspace(endT, startT, 1000)
    cte_int = cte_interpolated(temps)

    #change in length
    DeltaL = initL * np.trapz(cte_int, temps)

    finalL = initL - DeltaL

    return finalL, cte_int, temps

In [108]:
length = 10 #mm
startT = 298 #K
endT = 0.1 #K

results = []

materials = {'copper': copper_cte, 'silicon': silicon_cte}
for material, data in materials.items():
    finalL, cte, temps = lengthChangeWithTemp(material, data, length, startT, endT)
    plt.plot(temps, cte)
    print(material, ': ', finalL, '. : ', 100-finalL/length*100)
    results.append([material, finalL])

TypeError: tuple indices must be integers or slices, not str

In [ ]:
print(results)